<a href="https://colab.research.google.com/github/the-Quert/inlpfun/blob/master/Demo/LSTM/lstmGRU_GOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Mount GDrive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"stimastudio","key":"1d11bfc324c6e4d06be952e466654bf9"}\n'}

In [0]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

Source       
`First GOP Debate Twitter Sentiment`


In [0]:
'''
!kaggle datasets download -d crowdflower/first-gop-debate-twitter-sentiment
!unzip /content/first-gop-debate-twitter-sentiment.zip
!mv /content/Sentiment.csv /content/drive/My\ Drive/LAB/LSTM
'''

  0% 0.00/2.40M [00:00<?, ?B/s]
100% 2.40M/2.40M [00:00<00:00, 79.0MB/s]


In [0]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, GRU
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [0]:
data = pd.read_csv('/content/drive/My Drive/LAB/LSTM/Sentiment.csv')
data = data[['text', 'sentiment']]

In [0]:
data = data[data.sentiment != 'Neutral']
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[a-zA-Z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_features = 2000
# max_features = 3000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)


4472
16986


In [0]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 6, 128)            256000    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 6, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
embed_dim = 128
lstm_out = 196

model_gru = Sequential()
model_gru.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model_gru.add(SpatialDropout1D(0.4))
model_gru.add(GRU(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model_gru.add(Dense(2, activation='softmax'))
model_gru.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model_gru)

In [0]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(7188, 6) (7188, 2)
(3541, 6) (3541, 2)


In [0]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 12, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/12
 - 6s - loss: 0.5251 - accuracy: 0.7884
Epoch 2/12
 - 6s - loss: 0.5142 - accuracy: 0.7891
Epoch 3/12
 - 6s - loss: 0.5123 - accuracy: 0.7891
Epoch 4/12
 - 6s - loss: 0.5110 - accuracy: 0.7891
Epoch 5/12
 - 6s - loss: 0.5093 - accuracy: 0.7891
Epoch 6/12
 - 6s - loss: 0.5054 - accuracy: 0.7892
Epoch 7/12
 - 6s - loss: 0.4991 - accuracy: 0.7923
Epoch 8/12
 - 6s - loss: 0.4992 - accuracy: 0.7927
Epoch 9/12
 - 6s - loss: 0.4958 - accuracy: 0.7948
Epoch 10/12
 - 6s - loss: 0.4950 - accuracy: 0.7965
Epoch 11/12
 - 6s - loss: 0.4938 - accuracy: 0.7981
Epoch 12/12
 - 6s - loss: 0.4932 - accuracy: 0.7969


In [0]:
batch_size = 32
model_gru.fit(X_train, Y_train, epochs = 12, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/12
 - 6s - loss: 0.5225 - accuracy: 0.7881
Epoch 2/12
 - 6s - loss: 0.5165 - accuracy: 0.7891
Epoch 3/12
 - 6s - loss: 0.5120 - accuracy: 0.7891
Epoch 4/12
 - 6s - loss: 0.5079 - accuracy: 0.7892
Epoch 5/12
 - 6s - loss: 0.5012 - accuracy: 0.7905
Epoch 6/12
 - 6s - loss: 0.4982 - accuracy: 0.7945
Epoch 7/12
 - 6s - loss: 0.4993 - accuracy: 0.7938
Epoch 8/12
 - 6s - loss: 0.4965 - accuracy: 0.7962
Epoch 9/12
 - 6s - loss: 0.4946 - accuracy: 0.7986
Epoch 10/12
 - 6s - loss: 0.4945 - accuracy: 0.7962
Epoch 11/12
 - 6s - loss: 0.4953 - accuracy: 0.7965
Epoch 12/12
 - 6s - loss: 0.4937 - accuracy: 0.7983


#### Validation btn `LSTM` and `GRU`

In [0]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.49
acc: 0.80


In [0]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model_gru.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.52
acc: 0.79


#### Positive accuracy and Negative accuracy btn `LSTM` and `GRU`

In [0]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 5.47945205479452 %
neg_acc 93.54304635761589 %


In [0]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model_gru.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 7.534246575342466 %
neg_acc 99.17218543046357 %
